# 3. A ChatGPT Conversation with PyLapi

There are times when you want to have more control over API requests and responses, keep track of the API call sequence, or simply monitor the API interactions.
These tasks can be tedious and time-consuming. Your code may become more complex and therefore less readable.

To give an example, the OpenAI RESTful API is stateless, and each ChatGPT interaction is independent.
If you want to keep the conversation flowing, you need to keep track of the context.

For instance, you ask ChatGPT, "Where is the Sydney Opera House?" After receiving the answer, you ask a follow-up question: "How do I get *there*?"
To allow ChatGPT to work out that "there" means "the Sydney Opera House", you need to include the entire conversation in each API call.

## Keep the chat rolling

The `PyLapi` resource class below maintains a ChatGPT conversation flow by saving all questions and answers in the `Conversation` resource object, so whenever you call its `ask()` method, the entire conversation is sent to the OpenAI API, so its response will be context-aware.

The algorithm is handcrafted in just 39 lines, including space lines, without any automation or input from the OpenAPI specification, thanks to the rich features inherited from `PyLapi`.

In [1]:
from pylapi import PyLapi

class oAPI(PyLapi):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.api_url = "https://api.openai.com/v1"

@oAPI.resource_class(resource_name="conversation")
class Conversation(oAPI):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.message_bank = []

    @oAPI.resource_method(method_path="chat/completions", http_method="POST", load="$")
    def ask(self, question: str):

        @oAPI.callback
        def request(self, role="user", model="gpt-3.5-turbo", **kwargs):
            self.message_bank.append({
                "role": role,
                "content": kwargs["question"],
            })
            self.raw_request["json"] = {
                "model": model,
                "messages": self.message_bank,
                "temperature": 0.7
            }
            self.raw_request["params"] = {}

        @oAPI.callback
        def response(self, **kwargs):
            if self.response_ok() and "choices" in self.response_data:
                choices = self.response_data["choices"]
                self.message_bank.append(choices[0]["message"])
                answers = [_["message"]["content"] for _ in choices]
                self.response_data.update({
                    "answers": answers,
                })

The top-level API class `oAPI` inherits from `PyLapi` and sets the API URL `self.api_url`. The `Conversation` resource class inherits from `oAPI` and is decorated by `@oAPI.resource_class` with the resource name set to `"conversation"`.
The `__init__()` method creates an empty message bank.

The `@oAPI.resource_method` decorator makes the `ask()` method `POST` to the API path `/chat/completions`, and `load`s the entire response (as denoted by `$`) into the resource `data` attribute.

There are two `callback` methods:
- `request()` adds the user's question to the message bank before PyLapi sends it to ChatGPT.
- `response()` adds ChatGPT's answers to the message bank and also creates the "answers" data attribute.

---
Before trying it out, you need to [sign up with OpenAI](https://platform.openai.com/signup), obtain your [API key](https://platform.openai.com/account/api-keys), and save it to `._osecret` under the current directory for authenticating the API.

IMPORTANT: Please store your OpenAI API key securely without exposing it to any printouts, log messages, or repositories.

In [2]:
# Authenticate with your OpanAI API key previously saved in `._osecret`
oAPI.auth(open("._osecret", "r").readlines()[0].strip())

# Instantiate the conversation resource object
conversation = oAPI.resource("conversation")

## Let's talk!

Now you may ask your first question. Feel free to change the question to one that interests you.

In [3]:
conversation.ask("Where is the Sydney Opera House?")
print(conversation.data.answers[0])

The Sydney Opera House is located in Sydney, Australia. Specifically, it is situated on Bennelong Point in the Sydney Harbour, close to the Sydney Harbour Bridge.


When you ask a follow-up question, the `conversation` resource object will send the entire conversation from the beginning to ChatGPT so it is aware of the context.

In [4]:
conversation.ask("How to get there?")
print(conversation.data.answers[0])

There are several ways to get to the Sydney Opera House:

1. By Train: The closest train station to the Opera House is Circular Quay Station. From there, it's just a short walk to the Opera House.

2. By Ferry: Circular Quay is also a major ferry terminal, and many ferries stop there. You can take a ferry from various locations around Sydney, including Darling Harbour, Manly, or Watsons Bay, and disembark at Circular Quay.

3. By Bus: There are several bus routes that pass by or stop near the Opera House. You can check the local bus network for the best route to reach the Opera House from your location.

4. By Car: There is limited parking available at the Opera House, and it can be quite expensive. If you choose to drive, you can follow signs to the Sydney CBD (Central Business District) and then look for signs to the Opera House.

5. By Foot: If you are staying in the city center or near Circular Quay, you can easily walk to the Opera House. It's a scenic and enjoyable stroll along t

You may also give instructions to ChatGPT using the "system" role and alternatively specify the language model to use in the response.

In [5]:
conversation.ask("Speak like Harry Potter", role="system", model="gpt-3.5-turbo-16k")
print(conversation.data.answers[0])

To reach the Sydney Opera House, ye could use a variety of means:

1. By Floo Powder: Tis not available in the Muggle world, but if ye possess the magical ability, ye can use Floo Powder to transport yerself directly to the Opera House. Simply step into a fireplace, state "Sydney Opera House" clearly, and throw the powder into the flames. Be prepared for a dizzying journey!

2. By Broomstick: If ye're an accomplished wizard or witch with a trusty broomstick, soar through the skies like a true Quidditch player. Set a course for Sydney and navigate yer way to the Opera House. Remember to follow all broomstick safety guidelines and be mindful of Muggle airspace regulations.

3. By Portkey: Seek out a Portkey, a magically enchanted object that transports ye to a specific location when touched. Locate a Portkey that leads to the Sydney Opera House, grasp it tightly, and prepare for a swift and instantaneous journey.

4. By Apparition: For witches and wizards of age, Apparition is a handy sp

You may print the message bank at any time to review the conversation so far.

In [6]:
import json
print(json.dumps(conversation.message_bank, indent=2))

[
  {
    "role": "user",
    "content": "Where is the Sydney Opera House?"
  },
  {
    "role": "assistant",
    "content": "The Sydney Opera House is located in Sydney, Australia. Specifically, it is situated on Bennelong Point in the Sydney Harbour, close to the Sydney Harbour Bridge."
  },
  {
    "role": "user",
    "content": "How to get there?"
  },
  {
    "role": "assistant",
    "content": "There are several ways to get to the Sydney Opera House:\n\n1. By Train: The closest train station to the Opera House is Circular Quay Station. From there, it's just a short walk to the Opera House.\n\n2. By Ferry: Circular Quay is also a major ferry terminal, and many ferries stop there. You can take a ferry from various locations around Sydney, including Darling Harbour, Manly, or Watsons Bay, and disembark at Circular Quay.\n\n3. By Bus: There are several bus routes that pass by or stop near the Opera House. You can check the local bus network for the best route to reach the Opera House

## What's the point?

Beyond code simplification, the `PyLapi` resource model and callback features blur the line between frontend custom functions and backend API methods.

In the above example, the application calls the `conversation.ask()` method "naturally", without even knowing that it is not a native API method.

This can be very useful to emulate new API methods so your frontend developers can start using them even before they are released in the native API.

Moreover, there are functionalities that can be easily provided at the frontend, as shown in the example above. However, it can be very complicated to implement such functionalities at the backend, like passing handles (e.g., cookies) to and fro, storing the context in persistent storage, protecting the data in transit and at rest, etc.

---
So far, we have handcrafted the `oAPI` API class and the `Conversation` resource class.
If you want to automatically generate the `oAPI` API class with all OpenAPI methods, together with the custom `Conversation` resource class, `PyLapi` allows you to merge your custom code into the auto-generated code.

For demo purposes, we have put the custom `Conversation` class into [./oapi_rewrite.py](./oapi_rewrite.py) and configured [./oapi_config.py](./oapi_config.py) to merge the custom code into [./oapi.py](./oapi.py). Please go to the [first tutorial](./1.%20Getting%20Started%20with%20PyLapi.ipynb) for more details about the code-rewrite feature.

---
In the next tutorial, we are going to explore more [advanced PyLapi features](./4.%20PyLapi%20Advanced%20with%20Asana.ipynb) and see how it can make a programmer's life easier.

## End of page